In [153]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from google_auth_oauthlib import flow
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import requests
import json
import csv

SERVICE_ACCOUNT_FILE = 'key.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE)

bqclient = bigquery.Client(project='iotube-analytics', credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [217]:
COVALENT_KEY = "ckey_10446544a51944ffa7c6603a9e4"
chain = {
    "Polygon": 137,
    "Ethereum": 1,
    "BSC": 56
}
validator = {
    "Polygon": "0xFBe9A4138AFDF1fA639a8c2818a0C4513fc4CE4B",
    "Ethereum": "0xd8165188ccc135b3a3b2a5d2bc3af9d94753d955",
    "BSC": "0x116404F86e97846110EA08cd52fC2882d4AD3123"
}

In [218]:
def getBridgeDataIn(network):
    '''
    Gets inflow bridge data for a given network
    '''
    dfToIotex = pd.DataFrame(columns = ['Network', 'Date', 'Address', 'Transaction Fee', 'Token Name', 'Token Symbol', 'Value'])
    endpoint = "https://api.covalenthq.com/v1/{}/address/{}/transactions_v2/?page-size={}&key={}".format(chain[network], validator[network], 10000, COVALENT_KEY)
    response = requests.get(endpoint)
    JSON_DATA_RAW = response.json()
    JSON_DATA_RAW = JSON_DATA_RAW['data']

    for item in JSON_DATA_RAW['items']:
        temp = []
        temp.append(network)
        temp.append(item['block_signed_at'].split("T", 1)[0]) # Date
        temp.append(item['from_address']) # Address 
        temp.append(item['gas_price'] * item['gas_spent'] * pow(10, -18)) # Transaction Fee
        for event in item['log_events']:
            if(event['decoded']):
                if(event['decoded']['name'] == "Transfer"):
                    for param in event['decoded']['params']:
                        if(param["name"] == "to"):
                            address = param['value'] 
                        if(param["name"] == "value"):
                            value = param['value']
                    name = event['sender_name']
                    symbol = event['sender_contract_ticker_symbol']
                    decimals = -event['sender_contract_decimals']
                    value = int(value) * pow(10,decimals)
        temp.append(name) # Ticker Name
        temp.append(symbol) # Ticker Symbol
        temp.append(value) # Ticker Value
        dfToIotex.loc[len(dfToIotex)] = temp
                            
    dfToIotex['Date'] = pd.to_datetime(dfToIotex['Date'])
    return dfToIotex

In [225]:
polygon = getBridgeDataIn("Polygon")
ethereum = getBridgeDataIn("Ethereum")
bsc = getBridgeDataIn("BSC")
dfToIotex = polygon.append([ethereum, bsc])

In [228]:
inflow = pd.read_csv('https://storage.googleapis.com/iotube/bridgeInflowRaw')
inflow.head()

,Unnamed: 0,Network,Date,Address,Transaction Fee,Token Name,Token Symbol,Value
0,0,Polygon,2021-07-01,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0
1,1,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
2,2,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
3,3,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,5.0
4,4,Polygon,2021-06-17,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0


In [245]:
def crossChainGrouping(inDf):
    inflowDf = inDf.groupby(["Network", "Date"]).agg({'Value': ['sum', 'mean', 'median'],
                                                                'Transaction Fee': ['sum', 'mean', 'median']})
    inflowDf['Value']['sum'] = inflowDf['Value']['sum'].round()
    inflowDf['Transaction Fee']['sum'] = inflowDf['Transaction Fee']['sum']

    inflowDf = inflowDf.reset_index()
    inflowDf.columns = inflowDf.columns.droplevel(0)
    inflowDf.columns = ['Network', 'Date', 'Volume', 'Mean Volume', 'Median Volume', 'Transaction Fee', 'Mean Txn Fee', 'Median Txn Fee']
    
    return inflowDf

def tokenGrouping(inDf):
    dfPolygonInGroupedToken = inDf.groupby(["Token_Symbol", "Date"], as_index=False).agg({'Volume': ['count', 'sum', 'mean', 'median']})
    dfPolygonInGroupedToken['Volume']['sum'] = dfPolygonInGroupedToken['Volume']['sum'].round()

    dfPolygonInGroupedToken = dfPolygonInGroupedToken.reset_index()
    dfPolygonInGroupedToken.columns = dfPolygonInGroupedToken.columns.droplevel(0)
    dfPolygonInGroupedToken.columns = ['', 'Token_Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']

    dfPolygonInGroupedPeriod = inDf.groupby(["Bridge", "Token_Symbol", "Date"], as_index=False).agg({'Volume': ['count', 'sum', 'mean', 'median']})
    dfPolygonInGroupedPeriod['Volume']['sum'] = dfPolygonInGroupedPeriod['Volume']['sum'].round()

    dfPolygonInGroupedPeriod = dfPolygonInGroupedPeriod.reset_index()
    dfPolygonInGroupedPeriod.columns = dfPolygonInGroupedPeriod.columns.droplevel(0)
    dfPolygonInGroupedPeriod.columns = ['', 'Bridge', 'Token_Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']
    
    return dfPolygonInGroupedToken, dfPolygonInGroupedPeriod

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax.bar(dfPolygonInGrouped.index.get_level_values('Date'), dfPolygonInGrouped['Volume']['sum'], color='g', label='In Polygon')
ax.plot(dfPolygonInGrouped.index.get_level_values('Date'), dfPolygonInGrouped['Volume']['sum'], color='g')
ax.bar(dfPolygonOutGrouped.index.get_level_values('Date'), -dfPolygonOutGrouped['Volume']['sum'], color='red', label='To BSC')

fig.suptitle('Volume of total transfers BSC<>Polygon via EVO Bridge')
fig.supylabel('Volume')    

fig.legend()
plt.show()
fig.savefig('Plots/VolumeBy{}.jpg'.format(period))
plt.close()

In [ ]:
px.bar(dfPolygonInGrouped, 
        x=dfPolygonInGrouped['Date'], 
        y=dfPolygonInGrouped['Total'], 
        color=dfPolygonInGrouped['Bridge'],
       title="Volume of transfers into Polygon")

In [ ]:
import plotly.express as px

dfPolygonInGrouped = groupByTokenandPeriod(period = "Date")
dfPolygonInGrouped["In"] = "In Polygon"

vendors = dfPolygonInGrouped['Token Symbol']
regions = dfPolygonInGrouped.Bridge
count = dfPolygonInGrouped.Volume['count'].values
dates = dfPolygonInGrouped['Date'].values

df = pd.DataFrame(
    dict(vendors=vendors, regions=regions, dates = dates, count=count)
)

df["all"] = "all" # in order to have a single root node

fig = px.treemap(df, path=['all', 'regions', 'dates', 'vendors'], 
                 values='count', title="Total number of transactions between networks by token used")
fig.show()